# Cilia mapping of the whole dataset
The purpose of this notebook is to map all cilia from our dataset and visualise them with seaborn graphs. For this, we will concatanate all images in a single dataframe.

In [ ]:
from aicsimageio import AICSImage
from morphocilia.io import load_rescaled_channel
from morphocilia.classifier import scale_columns, classify_cilia
from morphocilia.segmenter import (
    cilia_segmenter_cleaner,
    get_region_map,
    segment_cd13,
    segment_opn,
)
from morphocilia.quantifier import cilia_props_table
from morphocilia.region_mapper import get_regions
from pathlib import Path
from skimage.measure import regionprops_table
import pandas as pd

In [ ]:
DATA_DIR = Path("../data")
filepaths = (
    DATA_DIR / "20231011_e18_cd13_opn_arl13.lif",
    DATA_DIR / "20231025_p1_cd13_opn_arl13b.lif",
    DATA_DIR / "20231122_p15_cd13_opn_arl13b.lif",
)

In [ ]:
scenes = []
for file in filepaths:
    if file.suffix != ".lif":
        continue

    file_handle = AICSImage(file)

    for scene in file_handle.scenes:
        print(scene)
        file_handle.set_scene(scene)
        if file_handle.dims["Z"][0] <= 1:
            continue

        print(file_handle.dims)
        scenes.append((file, scene))

In [ ]:
df_all = pd.DataFrame()
for image_path, scene in scenes:
    cilia_channel = load_rescaled_channel(
        image_path,
        scene,
        1,
    )
    opn_channel = load_rescaled_channel(
        image_path,
        scene,
        2,
    )
    cd13_channel = load_rescaled_channel(
        image_path,
        scene,
        3,
    )

    labelled_prediction = cilia_segmenter_cleaner(cilia_channel)
    opn_mask = segment_opn(opn_channel)
    cd13_mask = segment_cd13(cd13_channel)
    region_map = get_region_map(cd13_mask, opn_mask)

    this_df = cilia_props_table(labelled_prediction, cilia_channel)
    this_df["image_name"] = image_path
    this_df["scene_name"] = scene
    this_df[["region", "all_regions"]] = get_regions(
        labelled_prediction, region_map
    )[["region", "all_regions"]]

    df_all = pd.concat([df_all, this_df], ignore_index=True)
df_all

In [ ]:
def get_color(kind):
    if kind == "elongated":
        return 1
    elif kind == "fibroblastic":
        return 2
    elif kind == "looped":
        return 3
    elif kind == "uncertain":
        return 4
    else:
        return 5

In [ ]:
df_all[["scaled_solidity", "scaled_axis_minor_length"]] = scale_columns(
    df_all[["solidity", "axis_minor_length"]]
)

df_all["prediction"] = classify_cilia(
    df_all[["scaled_solidity", "scaled_axis_minor_length"]]
)

df_all["prediction_int"] = df_all.prediction.apply(get_color)

df_all["stage"] = df_all["image_name"].apply(lambda x: x.stem.split("_")[1])

df_all

In [ ]:
df_all.to_csv("cilia_mapping_all.csv", index=False)

In [ ]:
import seaborn as sns

In [ ]:
g = sns.FacetGrid(
    df_all.groupby(["stage", "scene_name", "region"])
    .prediction.value_counts()
    .to_frame()
    .reset_index(),
    col="region",
    row="prediction",
    sharey="row",
)
g.map_dataframe(sns.swarmplot, x="stage", y="count")

The results looks a bit odd. There seems to be no NA and CD13+ regions in E18 and P15 stages. We will need to check how the regionmap segmentation looks for those stages. Besides, there seems to be too many uncertain and fibroblastic cilia, which will make it problematic when normalising the cilia per region (done below).

In [ ]:
df_counts = (
    df_all.groupby(["stage", "scene_name", "region"])
    .prediction.value_counts()
    .to_frame()
    .reset_index()
)

a = df_counts.groupby(["stage", "scene_name", "region"])["count"].transform(
    "sum"
)
df_counts["relative_per_region"] = df_counts["count"].div(a)
df_counts

In [ ]:
g = sns.FacetGrid(
    df_counts, col="region", row="prediction", hue="scene_name", sharey="row"
)
g.map_dataframe(sns.swarmplot, x="stage", y="relative_per_region")
g.add_legend()

The colors seem to be repeated because some image names are the same for different stages. It only makes sense to look at them when looking at individual stages. In any case, it is too soon to draw any conclusions since there seems to be something wrong with the regionmap segmentation.